In [2]:
import sys; print(sys.executable)

/Users/sebastien/miniforge3/envs/klifs/bin/python


In [3]:
from bravado.client import SwaggerClient
from bravado.requests_client import RequestsClient

http = RequestsClient()
client = SwaggerClient.from_url(
    "https://klifs.net/swagger_v2/swagger.json",
    http_client=http,
    config={"also_return_response": False}
)

# Quick call: list AURKB
kinases = client.kinases.get_kinases_list(kinase_name="AURKB", species="Human").result()
print(kinases[0]["kinase.klifs_id"], kinases[0]["kinase.name"])

AttributeError: Resource kinases not found. Available resources: Information, Interactions, Ligands, Structures

In [4]:
import sys, pkgutil
print("Python:", sys.version)
print("Executable:", sys.executable)
print("Have importlib_resources?", "importlib_resources" in {m.name for m in pkgutil.iter_modules()})

import subprocess
subprocess.run([sys.executable, "-m", "pip", "show", "bravado"])
subprocess.run([sys.executable, "-m", "pip", "show", "swagger-spec-validator"])
subprocess.run([sys.executable, "-m", "pip", "show", "importlib_resources"])

Python: 3.11.13 | packaged by conda-forge | (main, Jun  4 2025, 14:52:34) [Clang 18.1.8 ]
Executable: /Users/sebastien/miniforge3/envs/klifs/bin/python
Have importlib_resources? True
Name: bravado
Version: 11.0.3
Summary: Library for accessing Swagger-enabled API's
Home-page: https://github.com/Yelp/bravado
Author: Digium, Inc. and Yelp, Inc.
Author-email: opensource+bravado@yelp.com
License: BSD 3-Clause License
Location: /Users/sebastien/miniforge3/envs/klifs/lib/python3.11/site-packages
Requires: bravado-core, monotonic, msgpack, python-dateutil, pyyaml, requests, simplejson, six, typing-extensions
Required-by: 
Name: swagger-spec-validator
Version: 3.0.4
Summary: Validation of Swagger specifications
Home-page: http://github.com/Yelp/swagger_spec_validator
Author: Yelp
Author-email: core-services@yelp.com
License: Apache License, Version 2.0
Location: /Users/sebastien/miniforge3/envs/klifs/lib/python3.11/site-packages
Requires: importlib-resources, jsonschema, pyyaml, typing-extensi

CompletedProcess(args=['/Users/sebastien/miniforge3/envs/klifs/bin/python', '-m', 'pip', 'show', 'importlib_resources'], returncode=0)

In [13]:
from bravado.client import SwaggerClient
from bravado.requests_client import RequestsClient
import pandas as pd
import numpy as np

# --- client (validation off) ---
http = RequestsClient()
client = SwaggerClient.from_url(
    "https://klifs.net/swagger_v2/swagger.json",
    http_client=http,
    config={
        "also_return_response": False,
        "validate_swagger_spec": False,
        "validate_responses": False,
        "validate_requests": False,
        "use_models": False,
    },
)

# --- ops we will use ---
kinase_id_op       = client.Information.get_kinase_ID
structures_list_op = client.Structures.get_structures_list

def call_autolist(op, **kwargs):
    specs = {p["name"]: p for p in op.operation.op_spec.get("parameters", [])}
    fixed = {}
    for k, v in kwargs.items():
        p = specs.get(k)
        ptype = p.get("type") if p else None
        if not ptype and p and isinstance(p.get("schema"), dict):
            ptype = p["schema"].get("type")
        if ptype == "array" and not isinstance(v, (list, tuple)):
            v = [v]
        fixed[k] = v
    return op(**fixed).result()

def kinase_id(kinase_name, species="Human"):
    rows = call_autolist(kinase_id_op, kinase_name=kinase_name, species=species)
    df = pd.DataFrame(rows)
    for col in ("kinase.klifs_id","kinase_ID","kinase.id","kinase.klifs_ID"):
        if col in df.columns:
            return int(df.iloc[0][col])
    id_like = [c for c in df.columns if "klifs" in c.lower() and "id" in c.lower()]
    return int(df.iloc[0][id_like[0]])

def structures_for_kinase(kid):
    rows = call_autolist(structures_list_op, kinase_ID=int(kid))
    return pd.DataFrame(rows)

def classify_from_pocket_strings(pocket_strings):
    """
    pocket_strings: list of 85-aa pocket strings (with '-' where absent in that structure)
    Returns: pandas Series indexed 1..85 with 'deletion/short-loop' or 'unresolved-in-some-structures'
    """
    # keep only valid 85-length strings
    seqs = [s for s in pocket_strings if isinstance(s, str) and len(s) == 85]
    if not seqs:
        raise ValueError("No valid 85-aa pocket strings found for this kinase.")

    # presence matrix: rows = 85 positions, cols = structures (True=present)
    M = np.column_stack([[c != "-" for c in s] for s in seqs])
    # deletion if absent in ALL structures at that position
    status = np.where(M.any(axis=1), "unresolved-in-some-structures", "deletion/short-loop")
    return pd.Series(status, index=pd.RangeIndex(1, 86), name="status")

# -------- run for your kinases --------
for kinase in ["AURKB", "BUB1"]:
    kid = kinase_id(kinase, species="Human")
    S = structures_for_kinase(kid)
    # your server provides the 85-aa pocket in S['pocket']
    pockets = S["pocket"].dropna().tolist()
    print(f"{kinase}: {len(pockets)} pocket strings found.")
    status = classify_from_pocket_strings(pockets)
    print(status.value_counts())
    deletions = sorted(status[status=="deletion/short-loop"].index.tolist())
    print("True deletions at KLIFS positions:", deletions)





AURKB: 1 pocket strings found.
status
unresolved-in-some-structures    85
Name: count, dtype: int64
True deletions at KLIFS positions: []
BUB1: 7 pocket strings found.
status
unresolved-in-some-structures    85
Name: count, dtype: int64
True deletions at KLIFS positions: []
